In [1]:
pip install pandas plotly dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [7]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Lecture des fichiers CSV
df_offre = pd.read_csv('structure_offre_TV.csv')
df_conso = pd.read_csv('structure_conso_TV.csv')

# Création de l'application Dash
app = dash.Dash(__name__)

# Liste des catégories (colonnes communes aux deux datasets)
categories = [col for col in df_offre.columns if col != 'Année']

# Layout de l'application
app.layout = html.Div([
    html.H1('Tableau de Bord TV - Offre vs Consommation (1998-2023)',
            style={'textAlign': 'center', 'marginBottom': 30}),

    # Sélecteur de catégorie
    html.Div([
        html.Label('Sélectionner une catégorie:'),
        dcc.Dropdown(
            id='category-selector',
            options=[{'label': cat, 'value': cat} for cat in categories],
            value='films'  # Valeur par défaut
        )
    ], style={'width': '50%', 'margin': 'auto', 'marginBottom': 30}),

    # Graphique d'évolution
    dcc.Graph(id='evolution-graph'),

    # Graphique des écarts
    dcc.Graph(id='gaps-graph')
])

# Callback pour mettre à jour le graphique d'évolution
@app.callback(
    Output('evolution-graph', 'figure'),
    [Input('category-selector', 'value')]
)
def update_evolution_graph(selected_category):
    fig = go.Figure()

    # Ajout de la courbe d'offre
    fig.add_trace(
        go.Scatter(
            x=df_offre['Année'],
            y=df_offre[selected_category],
            name='Offre',
            line=dict(color='#2563eb', width=2)
        )
    )

    # Ajout de la courbe de consommation
    fig.add_trace(
        go.Scatter(
            x=df_conso['Année'],
            y=df_conso[selected_category],
            name='Consommation',
            line=dict(color='#16a34a', width=2)
        )
    )

    # Mise à jour du layout
    fig.update_layout(
        title=f'Évolution de {selected_category}',
        xaxis_title='Année',
        yaxis_title='Pourcentage',
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    return fig

# Callback pour mettre à jour le graphique des écarts
@app.callback(
    Output('gaps-graph', 'figure'),
    [Input('category-selector', 'value')]
)
def update_gaps_graph(selected_category):
    # Calcul des écarts moyens
    gaps = []
    for cat in categories:
        avg_gap = (df_offre[cat] - df_conso[cat]).mean()
        gaps.append({'category': cat, 'gap': avg_gap})

    df_gaps = pd.DataFrame(gaps)

    # Création du graphique à barres
    fig = px.bar(
        df_gaps,
        x='category',
        y='gap',
        title='Écarts moyens entre offre et consommation',
        labels={'category': 'Catégorie', 'gap': 'Écart moyen'}
    )

    # Mise à jour du layout
    fig.update_layout(
        xaxis_tickangle=-45,
        showlegend=False,
        hovermode='x unified'
    )

    # Mise en évidence de la catégorie sélectionnée
    fig.update_traces(
        marker_color=[
            '#6366f1' if cat == selected_category else '#94a3b8'
            for cat in df_gaps['category']
        ]
    )

    return fig

# Fonction pour calculer des statistiques descriptives
def get_statistics(category):
    stats = {
        'moyenne_offre': df_offre[category].mean(),
        'moyenne_conso': df_conso[category].mean(),
        'max_offre': df_offre[category].max(),
        'max_conso': df_conso[category].max(),
        'min_offre': df_offre[category].min(),
        'min_conso': df_conso[category].min(),
        'ecart_moyen': (df_offre[category] - df_conso[category]).mean()
    }
    return stats

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
#source : https://www.pewresearch.org/journalism/fact-sheet/news-platform-fact-sheet/?tabItem=85f41328-b7d7-40b8-9b2d-7b06ff4201dc
# % of U.S. adults in each demographic group who get news at least sometimes from …
# September 17, 2024

In [4]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Lecture des données
habits_df = pd.read_csv('consumption_habits.csv')
media_df = pd.read_csv('media_consumption.csv')

# Fonction pour créer le graphique radar pour une tranche d'âge spécifique
def create_radar_plots(age_group):
    # Données traditionnelles
    traditional_data = habits_df[habits_df['Category'] == age_group].iloc[0]
    traditional_values = [
        traditional_data['Television'],
        traditional_data['Radio'],
        traditional_data['Print publications'],
        traditional_data['Digital devices']
    ]
    traditional_labels = ['Télévision', 'Radio', 'Publications Papier', 'Appareils Numériques']

    # Données numériques
    digital_data = media_df[media_df['Unnamed: 0'] == age_group].iloc[0]
    digital_values = [
        digital_data['News websites or apps'],
        digital_data['Social media'],
        digital_data['Search'],
        digital_data['Podcasts']
    ]
    digital_labels = ['Sites d\'actualités', 'Réseaux Sociaux', 'Recherche', 'Podcasts']

    # Créer le subplot avec deux graphiques radar
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'polar'}, {'type': 'polar'}]],
        subplot_titles=('Médias Traditionnels', 'Médias Numériques')
    )

    # Ajouter le radar des médias traditionnels
    fig.add_trace(
        go.Scatterpolar(
            r=traditional_values,
            theta=traditional_labels,
            fill='toself',
            name='Traditionnel',
            line_color='#8884d8'
        ),
        row=1, col=1
    )

    # Ajouter le radar des médias numériques
    fig.add_trace(
        go.Scatterpolar(
            r=digital_values,
            theta=digital_labels,
            fill='toself',
            name='Numérique',
            line_color='#82ca9d'
        ),
        row=1, col=2
    )

    # Mise à jour du layout
    fig.update_layout(
        title=f'Profil de Consommation Médiatique - {age_group}',
        showlegend=False,
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        polar2=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        width=1200,
        height=600
    )

    return fig

# Créer les graphiques pour chaque tranche d'âge
age_groups = ['Ages 18-29', '30-49', '50-64', '65+']

# Pour créer un graphique interactif pour toutes les tranches d'âge
for age in age_groups:
    fig = create_radar_plots(age)
    fig.show()

# Pour une version plus interactive avec un menu déroulant
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Profils de Consommation Médiatique par Âge',
            style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='age-selector',
        options=[{'label': age, 'value': age} for age in age_groups],
        value='Ages 18-29',
        style={'width': '50%', 'margin': '0 auto'}
    ),

    dcc.Graph(id='radar-graph')
])

@app.callback(
    Output('radar-graph', 'figure'),
    Input('age-selector', 'value')
)
def update_graph(selected_age):
    return create_radar_plots(selected_age)

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [5]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Lecture des données
traditional_df = pd.read_csv('media_preferences.csv')
alternative_df = pd.read_csv('alternative_media_preferences.csv')

# Configuration des couleurs
colors = {
    'Television': '#8884d8',
    'Radio': '#82ca9d',
    'Print publications': '#ffc658',
    'Digital devices': '#ff7c43',
    'News websites or apps': '#af7aa1',
    'Social media': '#ff9f40',
    'Search': '#59a14f',
    'Podcasts': '#4e79a7'
}

# Création de l'application Dash
app = Dash(__name__)

# Layout de l'application
app.layout = html.Div([
    html.H1('Préférences Médiatiques par Tranche d\'Âge',
            style={'textAlign': 'center', 'padding': '20px'}),

    # Boutons radio pour sélectionner le type de médias
    dcc.RadioItems(
        id='media-selector',
        options=[
            {'label': 'Tous les médias', 'value': 'all'},
            {'label': 'Médias traditionnels', 'value': 'traditional'},
            {'label': 'Médias alternatifs', 'value': 'alternative'}
        ],
        value='all',
        style={'textAlign': 'center', 'padding': '20px'},
        labelStyle={'padding': '0 10px'}
    ),

    # Le graphique
    dcc.Graph(id='media-graph')
])

@app.callback(
    Output('media-graph', 'figure'),
    Input('media-selector', 'value')
)
def update_graph(selected_view):
    # Préparation des données selon la vue sélectionnée
    if selected_view == 'traditional' or selected_view == 'all':
        trad_data = traditional_df.melt(
            id_vars=['Category'],
            value_vars=['Television', 'Radio', 'Print publications', 'Digital devices'],
            var_name='Media Type',
            value_name='Usage'
        )

    if selected_view == 'alternative' or selected_view == 'all':
        alt_data = alternative_df.melt(
            id_vars=['Category'],
            value_vars=['News websites or apps', 'Social media', 'Search', 'Podcasts'],
            var_name='Media Type',
            value_name='Usage'
        )

    # Création du graphique
    fig = go.Figure()

    # Ajout des barres selon la vue sélectionnée
    if selected_view in ['traditional', 'all']:
        for media_type in ['Television', 'Radio', 'Print publications', 'Digital devices']:
            data = trad_data[trad_data['Media Type'] == media_type]
            fig.add_trace(go.Bar(
                name=media_type,
                x=data['Category'],
                y=data['Usage'],
                marker_color=colors[media_type]
            ))

    if selected_view in ['alternative', 'all']:
        for media_type in ['News websites or apps', 'Social media', 'Search', 'Podcasts']:
            data = alt_data[alt_data['Media Type'] == media_type]
            fig.add_trace(go.Bar(
                name=media_type,
                x=data['Category'],
                y=data['Usage'],
                marker_color=colors[media_type]
            ))

    # Mise à jour du layout
    fig.update_layout(
        title='Utilisation des Médias par Tranche d\'Âge',
        xaxis_title='Tranche d\'Âge',
        yaxis_title='Pourcentage d\'utilisation',
        barmode='group',
        template='plotly_white',
        height=600,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    return fig

# Lancement de l'application
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
# % of U.S. adults in each demographic group who prefer ___ for getting news


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Lecture des données
traditional_df = pd.read_csv('media_preferences.csv')
alternative_df = pd.read_csv('alternative_media_preferences.csv')

# Harmonisation des catégories d'âge
traditional_df['Category'] = traditional_df['Category'].replace('18-29', 'Ages 18-29')

# Configuration des couleurs
colors = {
    'Television': '#8884d8',
    'Radio': '#82ca9d',
    'Print publications': '#ffc658',
    'Digital devices': '#ff7c43',
    'News websites or apps': '#af7aa1',
    'Social media': '#ff9f40',
    'Search': '#59a14f',
    'Podcasts': '#4e79a7'
}

# Création de l'application Dash
app = Dash(__name__)

# Layout de l'application
app.layout = html.Div([
    html.H1('Préférences Médiatiques par Tranche d\'Âge',
            style={'textAlign': 'center', 'padding': '20px'}),

    dcc.RadioItems(
        id='media-selector',
        options=[
            {'label': 'Tous les médias', 'value': 'all'},
            {'label': 'Médias traditionnels', 'value': 'traditional'},
            {'label': 'Médias alternatifs', 'value': 'alternative'}
        ],
        value='all',
        style={'textAlign': 'center', 'padding': '20px'},
        labelStyle={'padding': '0 10px'}
    ),

    dcc.Graph(id='media-graph')
])

@app.callback(
    Output('media-graph', 'figure'),
    Input('media-selector', 'value')
)
def update_graph(selected_view):
    # Préparation des données selon la vue sélectionnée
    if selected_view == 'traditional' or selected_view == 'all':
        trad_data = traditional_df.melt(
            id_vars=['Category'],
            value_vars=['Television', 'Radio', 'Print publications', 'Digital devices'],
            var_name='Media Type',
            value_name='Usage'
        )

    if selected_view == 'alternative' or selected_view == 'all':
        alt_data = alternative_df.melt(
            id_vars=['Category'],
            value_vars=['News websites or apps', 'Social media', 'Search', 'Podcasts'],
            var_name='Media Type',
            value_name='Usage'
        )

    # Création du graphique
    fig = go.Figure()

    # Ajout des barres selon la vue sélectionnée
    if selected_view in ['traditional', 'all']:
        for media_type in ['Television', 'Radio', 'Print publications', 'Digital devices']:
            data = trad_data[trad_data['Media Type'] == media_type]
            fig.add_trace(go.Bar(
                name=media_type,
                x=data['Category'],
                y=data['Usage'],
                marker_color=colors[media_type]
            ))

    if selected_view in ['alternative', 'all']:
        for media_type in ['News websites or apps', 'Social media', 'Search', 'Podcasts']:
            data = alt_data[alt_data['Media Type'] == media_type]
            fig.add_trace(go.Bar(
                name=media_type,
                x=data['Category'],
                y=data['Usage'],
                marker_color=colors[media_type]
            ))

    # Mise à jour du layout
    fig.update_layout(
        title='Utilisation des Médias par Tranche d\'Âge',
        xaxis_title='Tranche d\'Âge',
        yaxis_title='Pourcentage d\'utilisation',
        barmode='group',
        template='plotly_white',
        height=600,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    return fig

traditional_df['Category'] = traditional_df['Category'].replace('18-29', 'Ages 18-29')

# Lancement de l'application
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>